# Libraries

In [15]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy import stats
import warnings



warnings.filterwarnings('ignore')

# Loading Dataset

In [16]:
data = pd.read_excel("dataset.xlsx")

In [17]:
data.shape

(29, 2)

# EDA

In [18]:
data.head()

,Sentence,Outcome
0,What is the difference between depression and ...,C
1,What are other psychiatric conditions that can...,C
2,Why is depression more prevalent in women than...,C
3,What happens during menopause with regards to ...,C
4,What efforts are underway to improve treatment...,C


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  29 non-null     object
 1   Outcome   29 non-null     object
dtypes: object(2)
memory usage: 592.0+ bytes


In [20]:
for col in data.columns:
    print(data[col].value_counts())

What is the difference between depression and sadness?                                                                1
What are some of the treatments for depression?                                                                       1
What is Postpartum Depression (PPD)?                                                                                  1
What are some of the signs and symptoms of depression?                                                                1
What is depression?                                                                                                   1
What is postpartum depression?                                                                                        1
Can depression affect children?                                                                                       1
Who is at risk of depression?                                                                                         1
What are the causes of depression?      

In [21]:
data['Outcome'].value_counts()

P    8
C    7
I    7
O    7
Name: Outcome, dtype: int64

In [22]:
data.dtypes

Sentence    object
Outcome     object
dtype: object

In [23]:
data['Outcome'].value_counts()/np.float(len(data))

P    0.275862
C    0.241379
I    0.241379
O    0.241379
Name: Outcome, dtype: float64

In [24]:
print(round(data.describe(),4))

                                                 Sentence Outcome
count                                                  29      29
unique                                                 29       4
top     What is the difference between depression and ...       P
freq                                                    1       8


# Feature Vector and Target Variable

In [25]:
x=data.drop(["Outcome"],axis=1)
y=data["Outcome"]

In [32]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import re

In [33]:
word_corpus=data["Sentence"]
word_corpus

0     What is the difference between depression and ...
1     What are other psychiatric conditions that can...
2     Why is depression more prevalent in women than...
3     What happens during menopause with regards to ...
4     What efforts are underway to improve treatment...
5     Why have researchers so far failed to identify...
6     Could other health conditions be contributing ...
7                              Is depression treatable?
8              How is depression diagnosed and treated?
9     What is the most promising recent development ...
10    How can treatment-resistant depression be trea...
11    What ideas are on the horizon for better, more...
12      Can brain scans guide treatment for depression?
13           Do I need medicine to treat my depression?
14               Can depression affect physical health?
15      What are some of the treatments for depression?
16                 What about depression later in life?
17    How might depression affect my appetite an

In [37]:
words_only=re.sub("[^a-zA-Z]", " ",str(word_corpus))
print(words_only)

      What is the difference between depression and           What are other psychiatric conditions that can          Why is depression more prevalent in women than          What happens during menopause with regards to           What efforts are underway to improve treatment          Why have researchers so far failed to identify          Could other health conditions be contributing                                    Is depression treatable                 How is depression diagnosed and treated        What is the most promising recent development           How can treatment resistant depression be trea          What ideas are on the horizon for better  more            Can brain scans guide treatment for depression               Do I need medicine to treat my depression                   Can depression affect physical health          What are some of the treatments for depression                     What about depression later in life        How might depression affect my appetite an

In [38]:
words=words_only.lower().split()
print(words)

['what', 'is', 'the', 'difference', 'between', 'depression', 'and', 'what', 'are', 'other', 'psychiatric', 'conditions', 'that', 'can', 'why', 'is', 'depression', 'more', 'prevalent', 'in', 'women', 'than', 'what', 'happens', 'during', 'menopause', 'with', 'regards', 'to', 'what', 'efforts', 'are', 'underway', 'to', 'improve', 'treatment', 'why', 'have', 'researchers', 'so', 'far', 'failed', 'to', 'identify', 'could', 'other', 'health', 'conditions', 'be', 'contributing', 'is', 'depression', 'treatable', 'how', 'is', 'depression', 'diagnosed', 'and', 'treated', 'what', 'is', 'the', 'most', 'promising', 'recent', 'development', 'how', 'can', 'treatment', 'resistant', 'depression', 'be', 'trea', 'what', 'ideas', 'are', 'on', 'the', 'horizon', 'for', 'better', 'more', 'can', 'brain', 'scans', 'guide', 'treatment', 'for', 'depression', 'do', 'i', 'need', 'medicine', 'to', 'treat', 'my', 'depression', 'can', 'depression', 'affect', 'physical', 'health', 'what', 'are', 'some', 'of', 'the', '

In [40]:
print(stopwords.words("English"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [41]:
words_clean=[]
for word in words:
    if word not in stopwords.words("English"):
        words_clean.append(word)
print(words_clean)

['difference', 'depression', 'psychiatric', 'conditions', 'depression', 'prevalent', 'women', 'happens', 'menopause', 'regards', 'efforts', 'underway', 'improve', 'treatment', 'researchers', 'far', 'failed', 'identify', 'could', 'health', 'conditions', 'contributing', 'depression', 'treatable', 'depression', 'diagnosed', 'treated', 'promising', 'recent', 'development', 'treatment', 'resistant', 'depression', 'trea', 'ideas', 'horizon', 'better', 'brain', 'scans', 'guide', 'treatment', 'depression', 'need', 'medicine', 'treat', 'depression', 'depression', 'affect', 'physical', 'health', 'treatments', 'depression', 'depression', 'later', 'life', 'might', 'depression', 'affect', 'appetite', 'could', 'depression', 'affect', 'sleep', 'depression', 'cause', 'lack', 'concentration', 'could', 'depression', 'causing', 'physical', 'pain', 'causes', 'depression', 'risk', 'depression', 'depression', 'affect', 'children', 'postpartum', 'depression', 'depression', 'signs', 'symptoms', 'dep', 'postpa